In [1]:
import json
import re
from gensim.models import Word2Vec
import csv
import pandas as pd
from sklearn.cluster import DBSCAN

In [2]:
import snowballstemmer

In [3]:
f = open('digital_trajectoriesraject_public_keyword.json')
digital_trajectoriesraject_public_keyword = json.load(f)

f.close()

In [4]:
f = open('documents1.json')
documents = json.load(f)

f.close()

In [5]:
duplicates_data = pd.read_csv('duplicates_data.csv')
duplicates_data

,0,1,2
0,нереляционными субд,nosql базы данных,1
1,систем управления базами данных,системах управления базами данных,1
2,примере субд,система управления базами данных,1
3,разработке баз данных,реализации бд,1
4,субд,система управления базами данных,1
5,объектно-ориентированным программированием,ооп,1
6,бд,баз данных,1
7,big data,больших данных,1
8,использованием ros,применение пакетов ros,1
9,интеллектуального анализа данных,data mining,1


https://python-school.ru/blog/word2vec-with-examples-in-gensim/

## digital_trajectoriesraject_public_keyword

In [6]:
text = ""
for doc in documents: text += doc['text']

In [7]:
sentences = re.split(r'\.+\s', text.lower())


In [8]:
sentences[:3]

['коммерциализация результатов научных исследований и разработок представляет собой процесс вовлечения их в экономический (коммерческий) оборот в различных сегментах национального и глобального рынков',
 'актуальность данной дисциплины обусловлена становлением на путь модернизации экономики страны',
 'в сложившейся ситуации резко возрастает необходимость оценки и обоснования экономической эффективности проектов, ориентированных на выпуск высокотехнологичной продукции и продвижение новых технологий']

In [9]:
sentences_word_by_word = []
for i in sentences:
    i = re.sub(r'[^а-яёa-z -+.\/0-9]', '', i)
    i = re.sub(r'\xa0', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    sentences_word_by_word.append(i.split(' '))

In [10]:
arr = []
for i in digital_trajectoriesraject_public_keyword:
    i = i['value']
    i = re.sub('[^а-яёa-z -+.\/0-9]', '', i.lower())
    i = re.sub(r'\xa0', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    arr.append(i)
    
digital_trajectoriesraject_public_keyword = list(set([] + arr))

len(digital_trajectoriesraject_public_keyword), digital_trajectoriesraject_public_keyword[:5]

(21727,
 ['цели дисциплины изучение особенностей',
  'идей',
  'методы анализа погрешностей',
  'фундаментом',
  'порядок защиты вкр'])

In [291]:
w2v_model = Word2Vec(
    min_count=1,
    window=5,
    negative=0,
    alpha=0.5,
    min_alpha=0.005,
    sample=1e-4,
    sg=1,
    hs=1)

#hs поменял вывод

In [292]:
w2v_model.build_vocab(sentences_word_by_word)

In [293]:
w2v_model.train(sentences_word_by_word, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

(54594974, 76629080)

In [294]:
w2v_model.init_sims(replace=True)


/tmp/ipykernel_17307/2897808894.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [296]:
#w2v_model.save('w2v_model.model')

In [297]:
#w2v_model = Word2Vec.load('w2v_model.model')

In [298]:
X_test = []
dictionary_test = {}
notInVocab_test = []

for s in duplicates_data.iloc[:,:2].values:
    for phrase in s:
        vec = 0
        flag = 0
        for word in phrase.split(' '):
            if not word in w2v_model.wv.key_to_index.keys():
                flag = 1
                notInVocab_test.append(word)
                break
            vec += w2v_model.wv[word]
        vec /= len(phrase.split(' '))
        if flag:
            continue
        string = " ".join(str(v) for v in vec)
        if string in dictionary_test.keys():
            dictionary_test[string].append(phrase)
        else:
            dictionary_test[string] = [phrase]
        X_test.append(vec)
    
X_test[0]

array([ 0.00628512, -0.02137481, -0.03198995,  0.04108173,  0.04701893,
       -0.02191176, -0.03298412,  0.02062197, -0.04295439,  0.09006703,
       -0.03239756,  0.03810363, -0.02782618,  0.01421196, -0.15309621,
        0.05443034, -0.01730716,  0.00655147,  0.00692676,  0.13905044,
       -0.03840528, -0.07591582,  0.0463644 ,  0.10444763,  0.08393682,
       -0.08937322, -0.01082037, -0.10828389, -0.2034553 , -0.02650932,
       -0.01095977, -0.12521157,  0.0334976 ,  0.05287517,  0.0690845 ,
        0.05959144,  0.00539793, -0.06515025,  0.09194969,  0.04911564,
        0.02742365, -0.08956362, -0.12008783, -0.04437529,  0.02247142,
       -0.10901041,  0.03852612,  0.09951158,  0.03103765,  0.07495555,
       -0.03242134,  0.01296552,  0.08190592,  0.12843116,  0.09573746,
        0.04147454,  0.01559991,  0.06920205,  0.02288088,  0.06278261,
       -0.12600467, -0.02142962, -0.07496211,  0.01726194,  0.01925949,
       -0.02594102, -0.00951553, -0.01866102,  0.12301612,  0.13

In [299]:
import numpy as np
data_len = len(X_test) // 2
arr1 = []
arr0 = []
for i in range(data_len):
    point_1 = X_test[i * 2]
    point_2 = X_test[i * 2 + 1]
    distance = np.sqrt(sum((a-b) ** 2 for a, b in zip(point_1, point_2)))
    if duplicates_data.iloc[i, 2] == 1:
        arr1.append(distance)
    else:
        arr0.append(distance)


In [300]:
min1 = round(min(arr1), 4)
max1 = round(max(arr1), 4)
mean1 = round(np.array(arr1).mean(), 4)
med1 = round(np.median(np.array(arr1)), 4)

print('min =', min1, 
      'max =', max1, 
      'mean =', mean1,
      'median =', med1)

arr1, sorted(arr1)


min = 0.1588 max = 1.6096 mean = 0.8318 median = 0.8889


([0.3247121278158593,
  0.2003565461924676,
  0.8046616503673855,
  0.7544228469674107,
  1.1061975982091552,
  1.6096021375480967,
  1.2263547497899518,
  0.7619798285592533,
  0.45981607426910337,
  1.0083670363977524,
  1.1952997532547043,
  0.15877355439274965,
  0.9731426714973506,
  1.0621331677360109],
 [0.15877355439274965,
  0.2003565461924676,
  0.3247121278158593,
  0.45981607426910337,
  0.7544228469674107,
  0.7619798285592533,
  0.8046616503673855,
  0.9731426714973506,
  1.0083670363977524,
  1.0621331677360109,
  1.1061975982091552,
  1.1952997532547043,
  1.2263547497899518,
  1.6096021375480967])

In [301]:
min0 = round(min(arr0), 4)
max0 = round(max(arr0), 4)
mean0 = round(np.array(arr0).mean(), 4)
med0 = round(np.median(np.array(arr0)), 4)

print('min =', min0, 
      'max =', max0, 
      'mean =', mean0,
      'median =', med0)

arr0, sorted(arr0)

min = 0.3962 max = 1.5317 mean = 0.9127 median = 0.917


([0.9726085051759867,
  0.7126646863497473,
  0.8107518092338942,
  0.39618609591093834,
  1.5316840274861496,
  0.6659725858304409,
  0.9169718110479123,
  1.115213351514908,
  0.733462641230132,
  0.8114710072609485,
  1.0434558402314669,
  1.1776853278949415,
  0.9731594873806863,
  1.0463652790912281,
  0.7832806727537776],
 [0.39618609591093834,
  0.6659725858304409,
  0.7126646863497473,
  0.733462641230132,
  0.7832806727537776,
  0.8107518092338942,
  0.8114710072609485,
  0.9169718110479123,
  0.9726085051759867,
  0.9731594873806863,
  1.0434558402314669,
  1.0463652790912281,
  1.115213351514908,
  1.1776853278949415,
  1.5316840274861496])

In [280]:
print('min0-min1=', round(min0 - min1, 4), 
      '\nmax0-max1=', round(max0 - max1, 4), 
      '\nmean0-mean1=', round(mean0 - mean1, 4), 
      '\nmedian0-median1=', round(med0 - med1, 4))

min0-min1= 0.1827 
max0-max1= 0.1238 
mean0-mean1= 0.0103 
median0-median1= -0.0904


## При вычислении среднего из векторов

### 1


## При суммировании векторов

### Вариант n
при 

    min_count=1,
    window=5,
    negative=0,
    alpha=0.7,
    min_alpha=0.01,
    sample=1e-1,
    sg=1,
    hs=1
    
w2v_model.train(sentences_word_by_word, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

для 1: min = 0.2162 max = 3.7256 mean = 1.9104 median = 1.9625

для 0: min = 0.9745 max = 3.6274 mean = 2.0252 median = 2.1101

**0.2162** бд - баз данных

**3.7256** нереляционными субд	nosql базы данных	
 
 
**0.9745** оценку структуры	другие методы

**3.6274** систем моделирования	методы исследования свойств клеток


### Вариант №m

min0-min1= 0.3275

max0-max1= 0.6789 

mean0-mean1= 0.3233 

median0-median1= 0.1946

w2v_model = Word2Vec(

    min_count=1,
    window=5,
    negative=0,
    alpha=0.5,
    min_alpha=0.005,
    sample=1e-4,
    sg=1,
    hs=1)
    
w2v_model.train(sentences_word_by_word,total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

1: min = 0.6212 max = 2.8116 mean = 1.7783 median = 1.7289

0: min = 0.9487 max = 3.4905 mean = 2.1016 median = 1.9235

In [281]:
X = []
dictionary = {}
notInVocab = []

for phrase in digital_trajectoriesraject_public_keyword:
    vec = 0
    flag = 0
    #print(phrase)
    for word in phrase.split(' '):
        if not word in w2v_model.wv.key_to_index.keys():
            flag = 1
            notInVocab.append(word)
            break
        vec += w2v_model.wv[word]
    vec /= len(phrase.split(' '))
    if flag:
        continue
    string = " ".join(str(v) for v in vec)
    if string in dictionary.keys():
        dictionary[string].append(phrase)
    else:
        dictionary[string] = [phrase]
    X.append(vec)
    
X[0]

array([ 0.07667965, -0.07623653, -0.03359093,  0.00220814, -0.03954361,
       -0.01282498,  0.04993647,  0.0088951 , -0.03570224, -0.00232242,
        0.00416996,  0.09869556, -0.04086344, -0.10020621,  0.05532472,
       -0.00701046, -0.11598358, -0.04198572, -0.01518202,  0.01736334,
        0.0983262 ,  0.01114234, -0.04713894, -0.00266535, -0.03326277,
       -0.10458721,  0.12575072, -0.0016776 ,  0.10232102,  0.00189006,
        0.04002666,  0.00260146, -0.02732572, -0.07634328, -0.06496286,
        0.0388921 , -0.05948248,  0.07589099,  0.005123  ,  0.03323925,
       -0.02542162, -0.00169042,  0.05657913,  0.02592171,  0.03774942,
       -0.07337196, -0.05185881, -0.10319833, -0.0627277 , -0.01259799,
       -0.03833484,  0.06072747,  0.04713861,  0.02800905, -0.10122596,
        0.00083664,  0.03588481, -0.04448243,  0.02234865, -0.06421846,
        0.06654371,  0.11591928,  0.09429631,  0.01417029, -0.00509984,
       -0.05033511, -0.0199657 ,  0.00187691, -0.03014098, -0.00

In [282]:
#for i in list(dictionary.keys())[:1]:
#    print(i, dictionary[i], '\n\n')

0.076679654 -0.07623653 -0.033590928 0.0022081374 -0.039543614 -0.012824979 0.04993647 0.0088951 -0.035702236 -0.0023224228 0.0041699614 0.09869556 -0.04086344 -0.10020621 0.05532472 -0.0070104552 -0.11598358 -0.04198572 -0.015182016 0.017363343 0.0983262 0.0111423405 -0.047138937 -0.00266535 -0.033262767 -0.10458721 0.12575072 -0.0016776025 0.10232102 0.0018900572 0.040026665 0.0026014575 -0.02732572 -0.07634328 -0.064962864 0.038892098 -0.059482478 0.07589099 0.005123001 0.033239245 -0.025421623 -0.0016904203 0.05657913 0.025921712 0.03774942 -0.07337196 -0.05185881 -0.103198335 -0.062727705 -0.012597987 -0.03833484 0.06072747 0.04713861 0.028009046 -0.10122596 0.0008366434 0.03588481 -0.04448243 0.02234865 -0.06421846 0.066543706 0.11591928 0.094296314 0.01417029 -0.0050998367 -0.05033511 -0.019965695 0.0018769088 -0.030140981 -0.0049342583 -0.034656767 0.020526312 -0.089926414 0.0017914437 0.117303625 0.023426097 -0.02123697 -0.05558768 -0.041910924 0.06687929 0.04891429 0.00255165

### DBSCAN

In [342]:
clustering = DBSCAN(eps=0.05, min_samples=2, metric='cosine').fit(X)


In [343]:
clustering.labels_

array([  0,   1,  -1, ...,   1, 150,   1])

In [344]:
for i in range(-1, 20):
    print(list(clustering.labels_).count(i))

6457
2
13034
3
6
4
3
2
4
2
2
3
6
3
2
2
2
18
3
2
2


In [345]:
number_of_clusters = clustering.labels_.max()
synonyms_array = []
size = len(X)
labels = list(clustering.labels_)

for i in range(number_of_clusters):
    cluster = np.where(clustering.labels_ == i)[0]
    synonyms = []
    for vec in cluster:
        synonyms.extend(dictionary[" ".join([str(v) for v in X[vec]])])
    synonyms_array.append(synonyms)
            

In [346]:
for i in range(len(synonyms_array)):
    print(i)
    for j in synonyms_array[i][:20]:
        print(j)
    if len(synonyms_array[i]) > 20:
        print('...')
    print()

0
цели дисциплины изучение особенностей
цели дисциплины изучение роли

1
идей
порядок защиты вкр
примеры реализации систем
области отображения информации
объект пародии г. грасса 
архитектура qnx
системы фибс
накопители информации
понятие баз данных
технологии xml
электронного декларирования
уравнения трансформатора
база систем управления
решение npтрудных задач
наборах данных
применения алгоритмов управления
навыки управления процессом эксплуатации су сээс
моделей электроэнергетических
перспективы применения 2d материалов
видео материалов
...

2
исследование тп изготовления
методы изготовления волноводов
методы создания материалов

3
устройств экб
основные интересы ца
устройств
оперативнокалендарное планирование
схемы пав устройств
основные принципы конструирования

4
использования возможностей функции format
законах излучения
анализ особенностей преобразования излучения
получение фундаментальных

5
средства снижения шума
средствах снижения шума
расчёт снижения потерь мощности

6
испо

In [ ]:
for i in list(dictionary.keys()):
    if len(dictionary[i]) > 1:
        print(dictionary[i], '\n')

In [38]:
digital_trajectoriesraject_public_keyword[:100]

['windows 2000',
 'лингвистические методы',
 'стандарты работы',
 'история развития ос',
 'содержание курса',
 'координаты состояний равновесия',
 'работоспособности пп',
 'навыки анализа устойчивости движения манипуляторов',
 'особенностями ведения переговоров',
 'физике лазеров',
 'возможность восстановления сигналов',
 'культурное развитие города',
 'нейронных моделей ндс',
 'оценки эффекта изменения свойств наночастиц',
 'рекуррентность концепта',
 'применение жк пвмс',
 'закрепления навыков',
 'языке ассемблера',
 'обзор курса',
 'разработка технологического',
 'навыками построения алгоритма управления манипулятором',
 'курс предназначен',
 'дословный перевод',
 'automation',
 'навык определения потребностей бизнеса',
 'операторы языка си',
 'области проектирования',
 'оптоэлектроника',
 'средства построения систем ии',
 'перспективы применения методов теории вероятностей',
 'синтеза наночастиц металлов',
 'основная задача системы',
 'computational science',
 'принципами разработк

In [43]:
words = ['бд', 'данны']
for i in digital_trajectoriesraject_public_keyword:
    for word in words:
        if word in i['value']:
            print(i['value'])
            continue

способы достижения непротиворечивости данных
модели непротиворечивости данных
нужна очистка данных
подготовки данных
дополнение данных
методики представления данных
методики преобразования баз данных
описание наборов данных rdf
модель данных rdf
методов наполнения данными графов знаний
сбор данных пациента
обработка данных диагностики
инструменты получения данных
основе данных диагностики пациента
прототипированию данных
данных анатомической
основы передачи данных
операции передачи данных
масштабируемыми субд
формы хранения данных
системы баз данных
архитектуры серверов баз данных
данных изображений
изучение структур данных
оптимальных структур данных
иерархические структуры данных
организацией структуры данных
проблема обработки данных
обработки данных -
задачах обработки данных
моделирования данных
место баз данных
практике - тенденции развития технологий баз данных
реляционных баз данных
ведения систем баз данных
понятий теории баз данных
проектированию структуры базы данных
системн

In [73]:
for i in digital_trajectoriesraject_public_keyword[5200:5300]:
    print(i['value'])
    

надежного распределения электроэнергии
современных систем передачи
основы регулирования режимов систем передачи
распределения электрической
элементы проектирования систем распределения
регулирование напряжения изменением сопротивления сети
характеристика систем распределения
линии
проведении анализа режимов работы линий электропередач
электрических систем
энергии
современных способах передачи
сетей систем электроснабжения
редакторе power
нестандартного решения
оптимального решения
выполнение работы - поиск решения проблемы
нестандартных задач
нестандартных технических задач
изобретательских задач
поиска решения задач предприятий
майнд мэп - диаграммы связей
количество источников -
аналогия - сравнение
страниц -
бизнес-модель -
понимание методики выполнения работы
метод создания фильма
поиск решений
принцип действия машин
принцип действия двигателя
основные виды машин
регулирование скорости вращения двигателей
уравнения напряжения трансформатора
развития предприятия
государственного рег

### Мусор


In [18]:
len(X)

21627

In [52]:
len(notInVocab), notInVocab

(100,
 ['а.',
  'а.',
  'им.',
  'expressg.',
  'b.',
  'б.',
  'n.',
  'а.',
  'r.',
  '3g.',
  'м.д.с.',
  'а.',
  'вт.',
  'microstructured',
  'и.',
  'p.',
  'б.',
  'а.',
  'фишер',
  'э.',
  'lm.',
  'ч.',
  'ф.',
  'в2с.',
  'а.к.',
  'а.',
  'райт',
  'а.',
  'а.',
  'processor',
  'b.',
  'т.',
  'а.',
  'в.и.',
  'б.',
  'б.',
  'т.',
  'м.д.с.',
  'т.ч.',
  'f1(x',
  'а.',
  '5s.',
  'http(s',
  'maec',
  'ком.',
  'f1(x',
  'а.',
  'б.',
  '3d.',
  'languages',
  'а.',
  'geometry',
  'asi',
  'им.',
  'd.',
  'д.и.',
  'к.',
  'м.ю.',
  'а.',
  'l.',
  'ф.и.о.',
  'тыс.',
  'i.',
  'д.и.',
  'и.в.',
  'д.и.',
  'а.',
  'а.',
  'г.п.',
  'криптопроколов',
  'т.ч.',
  'а.',
  'прожект',
  'pow',
  '4d.',
  'б.',
  '#',
  'у.',
  'им.',
  'б.',
  'им.',
  'вс.',
  'c(x',
  'в2b.',
  'исон',
  'mdd',
  'f(x)x8+x6+x3',
  'f.',
  'а.',
  'а.',
  'а.',
  'kellogg',
  'e.',
  'ленина',
  'а.',
  'б.',
  'а.',
  'а.',
  'b.',
  'вс.'])